# Install required libraries

In [ ]:
!pip install azure-storage-blob
# or !pip install --upgrade --force-reinstall azure.storage.blob
# remember to restart kernel / pip list to confirm

# Establish blob connection

### The following code establishes connection with Azure Blob storage

In [48]:
# Check the version of the client library
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__
from datetime import datetime, timedelta

print("Azure Blob Storage v" + __version__) # should be at least Azure Blob Storage v12.9.0

Azure Blob Storage v12.9.0


# Please update these variables - 
###### Only If you are running this notebook separately, otherwise these parameters are picked up from main file

### storage_conn_string "Storage account connection string"
### src_container "Container where data is stored"
### dst_container "Container where results should be uploaded"

In [2]:
# # set up connection
# storage_conn_string = ""
# src_container = ""
# dst_container = ""

In [4]:
#or on linux, do: export AZURE_STORAGE_CONNECTION_STRING="<yourconnectionstring>" and then do storage_conn_string = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

# Create the BlobServiceClient object which will be used to create a container client
blob_service_client = BlobServiceClient.from_connection_string(storage_conn_string)

# create a client for a specific container
srcblob_container_client = blob_service_client.get_container_client(src_container)

# create a client for a specific container
dstblob_container_client = blob_service_client.get_container_client(dst_container)

# Download data from Blob

### The following function downloads data from blob storage to local folder
#### parameters

local_path         "Local path where data should be stored"


In [70]:
from pathlib import Path

def download2local(local_path, container_client = srcblob_container_client, container_name = src_container):

    # Create a local directory to hold blob data
    Path(local_path).mkdir(parents=True, exist_ok=True)

    # download all the files to the local_path folder
    blob_list = container_client.list_blobs() # need to run this again, no way to reset the iterator
    for blob in blob_list:

        # note that this is called on the service_client and not the container_client
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob)

        download_file_path = os.path.join(local_path, blob.name)
        print("\tDownloading " + blob.name + " to " + download_file_path)

        basedir = os.path.split(download_file_path)[0]
        Path(basedir).mkdir(parents=True, exist_ok=True)

        with open(download_file_path, "wb") as download_file:
            download_file.write(blob_client.download_blob().readall())

    print("Files Downloaded.")

In [ ]:
#local_path = "../data/samples"
#download2local(local_path)

# Upload data to Blob container

### The following function uploads data from local drive to azure blob storage container
#### parameters

local_path         "Local path where data should be stored" 
container_path     "Path on container where data should be stored"


In [38]:
from pathlib import Path
from os.path import relpath
import glob

def upload2blob(local_path, container_path = "", container_client = dstblob_container_client, container_name = dst_container):

    # list the local files
    files_to_upload = glob.glob(os.path.join(local_path,"**/*"), recursive=True)

    for file_to_upload in files_to_upload:
        print("\nUploading to Azure Storage as blob: \t" + file_to_upload)

        # Upload the file
        #path_to_file = os.path.join(local_path, file_to_upload)
        path_to_file = file_to_upload
        file_path_on_azure = relpath(file_to_upload, local_path)
        file_path_on_azure = os.path.join(container_path,file_path_on_azure)

        #print("local path:", path_to_file, "Azure path:", file_path_on_azure)
        # Create a blob client using the local file name as the name for the blob
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_path_on_azure)

        path = Path(path_to_file)
        if path.is_file():
            with open(path_to_file, "rb") as data:
                blob_client.upload_blob(data, overwrite = True)

In [ ]:
# local_path = "../data/samples"
# upload2blob(local_path)

# Delete data in Blob container

### The following function deletes all data in azure blob storage container


In [38]:
from pathlib import Path
from os.path import relpath
import glob

def deleteContainerData(container_client = dstblob_container_client, container_name = dst_container):

    blob_list = container_client.list_blobs() # need to run this again, no way to reset the iterator
    for blob in blob_list:
        dstblob_container_client.delete_blob(blob=blob)


# Create SAS signature for blob container

### This container generates SAS URL for a Blob storage container


In [1]:
from azure.storage.blob import AccessPolicy, ContainerSasPermissions
from azure.storage.blob import generate_container_sas, generate_account_sas

def fr_get_sas_url(dst_container = dst_container):
    #permission = ContainerSasPermissions(read=True, write=True, delete=True, list=True)
    permission = ContainerSasPermissions.from_string(storage_conn_string)
    expiry=datetime.utcnow() + timedelta(hours=24)
    start=datetime.utcnow() - timedelta(minutes=1)

    access_policy = AccessPolicy(permission, expiry, start)

    sas_token = generate_container_sas(
        account_name = blob_service_client.account_name,
        container_name = dst_container,
        account_key = blob_service_client.credential.account_key,
        permission = permission,
        expiry = expiry,
        start = start
    )
    
    #return(sas_token)
    
    url = 'https://'+blob_service_client.account_name+'.blob.core.windows.net/'+dst_container+'?'+sas_token
    return(url)
    

In [ ]:
#sas_token = fr_get_sas_url()


In [ ]:
#sas_token